# Creating processes

In [39]:
using MPIClusterManagers
using Distributed
if procs() == workers()
    nranks = 4
    manager = MPIWorkerManager(nranks)
    addprocs(manager)
end

4-element Vector{Int64}:
 6
 7
 8
 9

In [2]:
@everywhere workers() begin
    using CSV
end

In [40]:
@everywhere workers() begin
    using MPI
    MPI.Init()
    comm = MPI.COMM_WORLD
    nranks = MPI.Comm_size(comm)
    rank = MPI.Comm_rank(comm)
    println("Hello, I am process $rank of $nranks processes!")
end


      From worker 7:	Hello, I am process 1 of 4 processes!
      From worker 9:	Hello, I am process 3 of 4 processes!
      From worker 6:	Hello, I am process 0 of 4 processes!
      From worker 8:	Hello, I am process 2 of 4 processes!


## Question 4

### General functions

In [41]:
## GEneral funtions
@everywhere function mandelbrot_set(c,iters)
    "Does n iteration fo the mandelbor set for c"
    z = 0 # Mandelbrot set start always from 0
    n = 0
    while abs(z) <= 2 && n<iters
        z = z^2 + c
        n += 1
    end
    return n
end


@everywhere function detector(c,iters)
    "Functions that return 1 if a number is in madelbrot set, and 0 if it is not"
    n = mandelbrot_set(c,iters)
    if n == iters ## If the number is inside we get 1
        return 1
    else ## If the number is outside we get 0
        return 0
    end
end

In [42]:

## IT DELETES ALL THE DATA, AND GENEREATES NEW CSV FILES
@everywhere workers() begin
    using CSV
    using MPI
    MPI.Init()
    comm = MPI.Comm_dup(MPI.COMM_WORLD)
    nranks = MPI.Comm_size(comm)
    rank = MPI.Comm_rank(comm)
    
    open("Mande_complex$rank.csv", "w") do csvfile
        CSV.write(csvfile, [], headers = ["Real", "Imaginary", "Detect", "Iterations"], append = true)
    end
end



In [43]:

@everywhere function Data_creator(lim_x,lim_y,rank, sample)
    "Stores all de data creating a mesh in a csv file with, real part, imaginary part, detection (1 if is from mandelbrot set) and the numbers of iterations"
    width, height = (sample, sample) # We will generate a total of 
    iters = 200 #The maximum height we will look into
    min_x = lim_x[1]
    max_x = lim_x[2]
    min_y = lim_y[1]
    max_y = lim_y[2]
    data = []
    open("Mande_complex$rank and $sample.csv", "a") do csvfile
        for x in range(0,width)
            real = min_x + (x/width)*(max_x - min_x)
            for y in range(0,height)
                imag = min_y + (y/height)*(max_y - min_y)
                c = Complex(real,imag)

                n = mandelbrot_set(c,iters)
                det = detector(c,iters)
                dict = (Real = real, Imaginary= imag, Detect= det, Iterations = n)
                push!(data,dict) 
            end
            CSV.write(csvfile, data, append = true)
            data = []
        end
    end
end

In [44]:
#High load imbalance
@everywhere workers() begin
    using MPI
    using CSV
    MPI.Init()
    comm = MPI.Comm_dup(MPI.COMM_WORLD)
    nranks = MPI.Comm_size(comm)
    rank = MPI.Comm_rank(comm)
    root = 0 
    samples = [2200,1800,1400,1000,800,200,100,60,20,10,5] ## Edit here for the number of samples (sample*sample^*4 = total number)
    for sample in samples
        if rank == 0
            local lim_x =(-2,-0.5)
            local lim_y =(-1,0)
            Data_creator(lim_x,lim_y,rank,sample)
        end

        if rank == 1
            local lim_x =(-0.5,1)
            local lim_y =(-1,0) 
            Data_creator(lim_x,lim_y,rank,sample)
        end

        if rank == 2
            local lim_x =(-2,-0.5)
            local lim_y =(0,1)
            Data_creator(lim_x,lim_y,rank,sample)
        end

        if rank == 3
            local lim_x =(-0.5,1)
            local lim_y =(0,1) 
            Data_creator(lim_x,lim_y,rank,sample)
        end
    end
            
end

In [ ]:
## Better parallelization, Area copmutation not implemented for this result type
@everywhere workers() begin
    using MPI
    using CSV
    MPI.Init()
    comm = MPI.Comm_dup(MPI.COMM_WORLD)
    nranks = MPI.Comm_size(comm)
    rank = MPI.Comm_rank(comm)
    root = 0 
    nrows = 80
    size_rows = 2/nrows
    lim_x = (-2,1)
    row = 0
    for i in 0:nrows

        local lim_y = (-1 + size_rows*(i-1),-1 +  size_rows*(i))

        if row == rank
            Data_creator(lim_x,lim_y,rank)
        end
        
        global row += 1

        if row == nranks
            global row = 0
        end
    end
end

In [ ]:
#24 minutes for creating 400.000.000 manderplot values with 2000 iterations